In [1]:
from constants import common



/Users/luke.williams/git-repositories/hmpps-integration-api/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
URL = 'https://api-dev.prison.service.justice.gov.uk/v3/api-docs'
URLS = [
#API Docs
  "https://manage-adjudications-api-dev.hmpps.service.justice.gov.uk/v3/api-docs",
  "https://assess-risks-and-needs-dev.hmpps.service.justice.gov.uk/v3/api-docs",
  "https://court-register-dev.hmpps.service.justice.gov.uk/v3/api-docs",
  "https://raw.githubusercontent.com/ministryofjustice/curious-API/main/curious-api-specification.yaml",
  "https://sign-in-dev.hmpps.service.justice.gov.uk/auth/v3/api-docs",
  "https://keyworker-api-dev.prison.service.justice.gov.uk/v3/api-docs",
#Typo in URL? "https://allocation-manager-staging.apps.live-1.cloud-platform.service.justice.gov.uk/api-docs/index.html",
#timeout  "https://community-api.test.probation.service.justice.gov.uk/v3/api-docs/Community%20API",
#site can't be reached "https://probation-offender-events-dev.hmpps.service.justice.gov.uk/swagger-ui.html",
  "https://probation-offender-search-dev.hmpps.service.justice.gov.uk/v3/api-docs",
  "https://api-dev.prison.service.justice.gov.uk/v3/api-docs",
  "https://offender-events-dev.prison.service.justice.gov.uk/v3/api-docs",
  "https://prisoner-offender-search-dev.prison.service.justice.gov.uk/v3/api-docs",
# Timeout  "https://offender-dev.aks-dev-1.studio-hosting.service.justice.gov.uk/v3/api-docs",
  "https://prison-register-dev.hmpps.service.justice.gov.uk/v3/api-docs",
  "https://hmpps-allocations-dev.hmpps.service.justice.gov.uk/v3/api-docs",
# 403 forbidden, wrong url "https://probation-teams-dev.prison.service.justice.gov.uk/swagger-ui/index.html",
  "https://hmpps-interventions-service-dev.apps.live-1.cloud-platform.service.justice.gov.uk/v3/api-docs",
  "https://restricted-patients-api-dev.hmpps.service.justice.gov.uk/v3/api-docs",
  "https://hmpps-staff-lookup-service-dev.hmpps.service.justice.gov.uk/v3/api-docs",
# 503 unavailable, wrong url "https://hmpps-tier-dev.hmpps.service.justice.gov.uk/swagger-ui.html",
  "https://token-verification-api-dev.prison.service.justice.gov.uk/v3/api-docs",
# 503, wrong url "https://hmpps-workload-dev.hmpps.service.justice.gov.uk/swagger-ui.html"

#Microservices
  "https://raw.githubusercontent.com/ministryofjustice/hmpps-complexity-of-need/main/Complexity%20Of%20Need%20API%20Specification.yaml"
]
SEARCH_TERM = 'sentence'
SCHEMA = ''

In [3]:
def retrieve_schema(dictionary, schema_name, partial_match=True):
    """
    TODO Documentation
    Output the nested _name object for a particular _name from a dictionary response object

        Parameters:

        Returns:
    """
    #TODO figure out a way of storing the collected data as a list or dataframe so this function can return an object of such data
    #json_schemas = []
    if dictionary == '{}':
        print("Problem with dictionary object")
    else:
    #Lazy error catching to skip
        try:
            for key in dictionary["components"]["schemas"]:
                if str.lower(key) == str.lower(schema_name): 
                    print("Returning " + key + " schema")
                    print(dictionary["components"]["schemas"][schema_name], '\n')
                    #json_schemas.append(dictionary["components"]["schemas"][schema_name])
                elif partial_match==True and str.lower(schema_name) in str.lower(key):
                    print("Returning " + key + " schema")
                    print(dictionary["components"]["schemas"][key], '\n')
                    #json_schemas.append(dictionary["components"]["schemas"][schema_name])
        except KeyError as ke:
            print(ke)
            pass
        except Exception as err:
            print(f"Unexpected {err=}, {type(err)=}")
            raise
    #return json_schemas

def findKeyword(dictionary, search_key):
    """
    Search for the existence of a key word within a complicated dictionary response object

        Parameters:

        Returns:

    """
    # searching the presence of each key in the dictionary
    if search_key in dictionary:
        return True
    else:
        # run a loop on all the keys in that level
        for key in dictionary:
            # check if the value of that key is a dictionary
            if isinstance(dictionary[key], dict):
                # recall the function to check for the searched key in the next level
                if findKeyword(dictionary[key], search_key):
                    return True
            if isinstance(dictionary[key], list):
                for element in dictionary[key]:
                    if isinstance(element, dict) and findKeyword(element, search_key):
                        return True
    return False

In [4]:
json_extract = common.extract_data(URL)

In [7]:
findKeyword(json_extract,"Sentence")

False